This notebook is used for training anomalous resource access model.  The data used here is File Share Access Events from Windows machine. Data is loaded from a Blob Storage Container.
The trained model is then saved to the Blob Storage, which can then be used by the Scoring Notebook

Steps:
   0. One-time: Install the following packages on your cluster by navigating to the 'Clusters' tab on the left
        - sentinel_ai (whl package from GitHub Utilities folder)
        - azure_sentinel_ml_utilities (whl package from GitHub Utilities folder)
        - azure-storage-blob (from PyPi)
        - scikit-surprise (from PyPi)
        - numpy==1.15.0 (from PyPi)
        - pyarrow==0.12.0 (from PyPi)
        
   1. One-time: Set credentials in KeyVault so the notebook can access 
        - Storage Account
   2. Ensure the relative paths to Blob Storage are correct.
   3. Run the Notebook to produce the model
   
 One-time: (Setting up Storage Key in KeyVault)
    - (Refer:- https://docs.databricks.com/spark/latest/data-sources/azure/azure-storage.html#access-azure-blob-storage-directly)
 
 Storing and retrieving secrets: 
    - Using Azure KeyVault:- https://docs.azuredatabricks.net/user-guide/secrets/secret-scopes.html#akv-ss

In [2]:
import datetime as dt

# Storage Account information
storage_account = 'YOUR STORAGE ACCOUNT HERE'
storage_key = dbutils.secrets.get(scope = 'NAME HERE', key = 'KEY NAME HERE')
container = 'CONTAINER NAME HERE'
mount_point_name = 'MOUNT POINT NAME HERE'

train_base_path = 'PATHNAME HERE'

# Project name
project = 'PROJECT NAME HERE'

###
### Note that when training periodically, you specify time range relative to current time as specified in the commented section
###
# Time range for training
# train_start_time = dt.datetime.now() - dt.timedelta(days=65)
# train_end_time = dt.datetime.now() - dt.timedelta(days=10)
train_start_time = dt.datetime.strptime('Dec 1 2018', '%b %d %Y') 
train_end_time = dt.datetime.strptime('Jan 20 2019', '%b %d %Y') 

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-896007305526709> in <module> () 
 3 # Storage Account information 
 4 storage_account = 'asirsa2019anilstoragews2' 
 ----> 5 storage_key = dbutils . secrets . get ( scope = 'asirsa2019' , key = 'asirsa2019anilstoragews2' ) 
 6 container = 'fileshareaccessdata' 
 7 mount_point_name = '/mnt/rootFSSTr/' 

 /local_disk0/tmp/1570131528280-0/dbutils.py in get (self, scope, key) 
 167 
 168 def get ( self , scope , key ) : 
 --> 169 return self . entry_point . getDbutils ( ) . preview ( ) . secret ( ) . get ( scope , key ) 
 170 
 171 def getBytes ( self , scope , key ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o221.get.
: com.databricks.common.client.DatabricksServiceException: PERMISSION_DENIED: User fafadavi@microsoft.com does not have READ permission on scope asirsa2019 to perform this action.
	at com.databricks.common.client.DatabricksServiceException.copy(DBHttpClient.scala:592)
	at com.databricks.common.client.RawDBHttpClient.getResponseBody(DBHttpClient.scala:547)
	at com.databricks.common.client.RawDBHttpClient.httpRequestInternal(DBHttpClient.scala:498)
	at com.databricks.common.client.RawDBHttpClient.entityEnclosingRequestInternal(DBHttpClient.scala:489)
	at com.databricks.common.client.RawDBHttpClient.getInternal(DBHttpClient.scala:454)
	at com.databricks.common.client.RawDBHttpClient.get(DBHttpClient.scala:266)
	at com.databricks.common.client.DBHttpClient.get(DBHttpClient.scala:196)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient$$anonfun$get$1.apply(SecretManagerClient.scala:153)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient$$anonfun$get$1.apply(SecretManagerClient.scala:153)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:369)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.withAttributionContext(SecretManagerClient.scala:76)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:271)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.withAttributionTags(SecretManagerClient.scala:76)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:350)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.recordOperation(SecretManagerClient.scala:76)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.com$databricks$backend$common$rpc$SimpleSecretManagerClient$$performRequest(SecretManagerClient.scala:97)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient$$anonfun$performRequestReliably$2.apply(SecretManagerClient.scala:115)
	at com.databricks.common.client.DBHttpClient$.retryWithDeadline(DBHttpClient.scala:133)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.performRequestReliably(Secret

In [3]:
###
### You can do this one-time in a separate Notebook, so that you don't cause accidental errors in other Notebooks mounting/unmounting the folder
###

# Mount the Storage Container
#    (Refer:- https://docs.databricks.com/spark/latest/data-sources/azure/azure-storage.html#mount-azure-blob-storage-containers-with-dbfs)
dbutils.fs.mount(
 source = "wasbs://" + container + "@" + storage_account + ".blob.core.windows.net",
 mount_point = mount_point_name,
 extra_configs = {"fs.azure.account.key." + storage_account + ".blob.core.windows.net":storage_key})

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526710> in <module> () 
 6 # (Refer:- https://docs.databricks.com/spark/latest/data-sources/azure/azure-storage.html#mount-azure-blob-storage-containers-with-dbfs) 
 7 dbutils.fs.mount(
 ----> 8 source = "wasbs://" + container + "@" + storage_account + ".blob.core.windows.net" , 
 9 mount_point = mount_point_name , 
 10 extra_configs = {"fs.azure.account.key." + storage_account + ".blob.core.windows.net":storage_key})

 NameError : name 'container' is not defined

In [4]:
import numpy as np
import pandas as pd

from pyspark.sql import functions as f, types as t
from pyspark.sql.functions import udf

# ML
from sentinel_ai.peer_anomaly import spark_collaborative_filtering as scf 

# spark
from sentinel_ai.utils import sparkutils

#utils
from azure_sentinel_ml_utilities.azure_storage import blob_manager

--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-896007305526711> in <module> () 
 12 
 13 #utils 
 ---> 14 from azure_sentinel_ml_utilities . azure_storage import blob_manager 

 /databricks/python/lib/python3.5/site-packages/azure_sentinel_ml_utilities/azure_storage.py in <module> () 
 2 # Class for accessing blobs on Storage 
 3 # 
 ----> 4 from azure . storage . blob import BlockBlobService 
 5 import datetime as dt 
 6 

 ImportError : No module named 'azure'

In [5]:
checkpoint_dir = mount_point_name + 'cache/{0}/checkpoints'.format(project)
dbutils.fs.mkdirs(checkpoint_dir)
sparkutils.set_checkpointdir(spark, checkpoint_dir)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526712> in <module> () 
 ----> 1 checkpoint_dir = mount_point_name + 'cache/{0}/checkpoints' . format ( project ) 
 2 dbutils . fs . mkdirs ( checkpoint_dir ) 
 3 sparkutils . set_checkpointdir ( spark , checkpoint_dir ) 

 NameError : name 'mount_point_name' is not defined

# Load Dataset

In [7]:
class FileShareDataset:
  
    def __init__(self, storage_account, storage_key):
      self.storage_account = storage_account
      self.blob_manager = blob_manager(storage_account, storage_key)
      # Spark conf set for spark.read.csv to work
      spark.conf.set(
        "fs.azure.account.key." + storage_account + ".blob.core.windows.net",
        storage_key)

    @staticmethod
    def get_schema():
      return t.StructType([
        t.StructField('Timestamp', t.TimestampType()),
        t.StructField('Actor', t.StringType()),
        t.StructField('Resource', t.StringType()),
        t.StructField('categoricalFeatures', t.StringType()),
        t.StructField('count_', t.IntegerType())
      ])

    @staticmethod
    def _make_days_delta():
      @udf('double')
      def days_delta(d2, d1):
        return 1.0 + (d2 - d1).days

      return days_delta

    def get_raw_df(self, start_time, end_time, container, root, use_schema=True):
        
      blob_names = self.blob_manager.enumerate_blob_names(start_time, end_time, container, root)
      full_blob_names = ["wasbs://" + container + "@" + self.storage_account + ".blob.core.windows.net/" + bn for bn in blob_names]
      
      schema = FileShareDataset.get_schema() if use_schema else None
      
      if use_schema:
          return spark.read.csv(full_blob_names, schema=schema, sep='\t', header=False)
      else:
          return spark.read.csv(full_blob_names, sep='\t', header=False)

    def processed_df(self, df):
        dd = FileShareDataset._make_days_delta()

        return df.select(
            f.col('Timestamp').alias('timestamp1'),
            f.col('Timestamp').alias('timestamp2'),
            'Actor',
            'Resource',
            'count_'
        ).groupBy(
            'Actor',
            'Resource'
        ).agg({
            'timestamp1': 'min',
            'timestamp2': 'max',
            'count_': 'sum'
        }).select(
            f.lit('0').alias('tid'),
            f.col('min(timestamp1)').alias('min_timestamp'),
            f.col('max(timestamp2)').alias('max_timestamp'),
            f.col('Actor').alias('user'),
            f.col('Resource').alias('res'),
            (f.col('sum(count_)')/dd(f.col('max(timestamp2)'), f.col('min(timestamp1)'))).alias('score')
        )

    def get_dataset(self, start_time, end_time, container, root):
        return self.processed_df(self.get_raw_df(start_time, end_time, container, root)).cache()

In [8]:
def getdataset():
  return FileShareDataset(storage_account, storage_key).get_dataset(train_start_time, train_end_time, container, train_base_path)

In [9]:
ptraining = getdataset()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526717> in <module> () 
 ----> 1 ptraining = getdataset ( ) 

 <command-896007305526716> in getdataset () 
 1 def getdataset ( ) : 
 ----> 2 return FileShareDataset ( storage_account , storage_key ) . get_dataset ( train_start_time , train_end_time , container , train_base_path ) 

 NameError : name 'storage_key' is not defined

In [10]:
print(ptraining.first())

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526718> in <module> () 
 ----> 1 print ( ptraining . first ( ) ) 

 NameError : name 'ptraining' is not defined

In [11]:
print(ptraining.select('tid').distinct().count())

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526719> in <module> () 
 ----> 1 print ( ptraining . select ( 'tid' ) . distinct ( ) . count ( ) ) 

 NameError : name 'ptraining' is not defined

In [12]:
ptraining.describe().show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526720> in <module> () 
 ----> 1 ptraining . describe ( ) . show ( ) 

 NameError : name 'ptraining' is not defined

In [13]:
sparkutils.df_stats(ptraining)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526721> in <module> () 
 ----> 1 sparkutils . df_stats ( ptraining ) 

 NameError : name 'ptraining' is not defined

# Build Model

In [15]:
# Model building
access_anomaly = scf.AccessAnomaly(tenant_colname='tid', score_colname='score')
access_anomaly_model = access_anomaly.fit(ptraining)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526723> in <module> () 
 1 # Model building 
 2 access_anomaly = scf . AccessAnomaly ( tenant_colname = 'tid' , score_colname = 'score' ) 
 ----> 3 access_anomaly_model = access_anomaly . fit ( ptraining ) 

 NameError : name 'ptraining' is not defined

# Save Model

In [17]:
model_output = '{root}/{project}/model_output'.format(root=mount_point_name + 'models/', project=project)
print(model_output)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526725> in <module> () 
 ----> 1 model_output = '{root}/{project}/model_output' . format ( root = mount_point_name + 'models/' , project = project ) 
 2 print ( model_output ) 

 NameError : name 'mount_point_name' is not defined

In [18]:
access_anomaly_model.save(
    '{model_output}/access_anomaly_model'.format(model_output=model_output)
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526726> in <module> () 
 ----> 1 access_anomaly_model.save(
 2 '{model_output}/access_anomaly_model' . format ( model_output = model_output ) 
 3 )

 NameError : name 'access_anomaly_model' is not defined

In [19]:
# unmount blob storage
dbutils.fs.unmount(mount_point_name)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-896007305526727> in <module> () 
 1 # unmount blob storage 
 ----> 2 dbutils . fs . unmount ( mount_point_name ) 

 NameError : name 'mount_point_name' is not defined